# Basic Decline Curve Analysis

This notebook demonstrates the fundamentals of decline curve analysis using Arps models.

## What You'll Learn
- Load and visualize production data
- Fit Arps decline models (exponential, harmonic, hyperbolic)
- Generate production forecasts
- Evaluate model performance
- Create professional visualizations

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from decline_analysis import dca

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Load Production Data

We'll start by loading sample production data from a single well.

In [ ]:
# Load sample well data
df = pd.read_csv('data/sample_well_data.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

# Display first few rows
print("Production Data Summary:")
print(f"Period: {df.index[0].strftime('%Y-%m')} to {df.index[-1].strftime('%Y-%m')}")
print(f"Data points: {len(df)}")
print(f"Peak production: {df['oil_bbl'].max():.0f} bbl/month")
print(f"Current production: {df['oil_bbl'].iloc[-1]:.0f} bbl/month")
print(f"Cumulative production: {df['oil_bbl'].sum():.0f} bbl")

df.head()

## 2. Visualize Historical Production

In [ ]:
# Plot production history
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(df.index, df['oil_bbl'], 'o-', linewidth=2, markersize=4, label='Historical Production')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Oil Production (bbl/month)', fontsize=12)
ax.set_title('Well Production History', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Fit Arps Decline Models

We'll compare three types of Arps decline models:
- **Exponential**: Constant decline rate (b=0)
- **Harmonic**: Decline rate decreases linearly (b=1)
- **Hyperbolic**: Decline rate decreases with exponent b (0<b<1)

In [ ]:
# Create series for forecasting
series = df['oil_bbl']

# Forecast with different Arps models
models = {
    'Exponential': 'exponential',
    'Harmonic': 'harmonic',
    'Hyperbolic': 'hyperbolic'
}

forecasts = {}
metrics = {}

for name, kind in models.items():
    # Generate 12-month forecast
    forecast = dca.forecast(series, model='arps', kind=kind, horizon=12)
    forecasts[name] = forecast
    
    # Evaluate on historical data
    metric = dca.evaluate(series, forecast)
    metrics[name] = metric
    
    print(f"\n{name} Model:")
    print(f"  RMSE: {metric['rmse']:.2f} bbl/month")
    print(f"  MAE: {metric['mae']:.2f} bbl/month")
    print(f"  SMAPE: {metric['smape']:.2f}%")

## 4. Compare Model Forecasts

In [ ]:
# Plot all forecasts together
fig, ax = plt.subplots(figsize=(14, 7))

# Plot historical data
ax.plot(series.index, series.values, 'o-', 
        color='blue', linewidth=2, markersize=5, label='Historical', zorder=5)

# Plot forecasts
colors = ['red', 'green', 'orange']
for (name, forecast), color in zip(forecasts.items(), colors):
    # Extract forecast portion only
    forecast_part = forecast.iloc[len(series):]
    ax.plot(forecast_part.index, forecast_part.values, '--', 
            color=color, linewidth=2, label=f'{name} Forecast', alpha=0.8)

# Add vertical line at forecast start
ax.axvline(x=series.index[-1], color='gray', linestyle=':', linewidth=2, label='Forecast Start')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Oil Production (bbl/month)', fontsize=12)
ax.set_title('Arps Model Comparison', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Model Performance Comparison

In [ ]:
# Create metrics comparison DataFrame
metrics_df = pd.DataFrame(metrics).T
print("\nModel Performance Metrics:")
print(metrics_df.round(2))

# Find best model
best_model = metrics_df['rmse'].idxmin()
print(f"\n✓ Best performing model: {best_model} (lowest RMSE)")

In [ ]:
# Visualize metrics comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, metric in enumerate(['rmse', 'mae', 'smape']):
    metrics_df[metric].plot(kind='bar', ax=axes[idx], color=['red', 'green', 'orange'])
    axes[idx].set_title(f'{metric.upper()} Comparison', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel(metric.upper())
    axes[idx].set_xlabel('Model')
    axes[idx].grid(True, alpha=0.3, axis='y')
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6. Generate Final Forecast with Best Model

In [ ]:
# Use best model for final forecast
best_kind = models[best_model]
final_forecast = dca.forecast(series, model='arps', kind=best_kind, horizon=24)

print(f"Final Forecast using {best_model} Model:")
print(f"\nForecast for next 6 months:")
forecast_only = final_forecast.iloc[len(series):len(series)+6]
for date, value in forecast_only.items():
    print(f"  {date.strftime('%Y-%m')}: {value:.0f} bbl/month")

# Calculate decline rate
current_rate = series.iloc[-1]
forecast_6m = forecast_only.iloc[5]
decline_pct = ((current_rate - forecast_6m) / current_rate) * 100
print(f"\nExpected 6-month decline: {decline_pct:.1f}%")

## 7. Professional Visualization

In [ ]:
# Create publication-ready plot
dca.plot(series, final_forecast, 
         title=f'Production Forecast - {best_model} Arps Model',
         filename='production_forecast.png')

print("✓ Plot saved as 'production_forecast.png'")

## Summary

In this notebook, we:
1. Loaded and visualized production data
2. Fitted three types of Arps decline models
3. Generated and compared forecasts
4. Evaluated model performance
5. Selected the best model based on metrics
6. Created professional visualizations

## Next Steps

- **Notebook 02**: Learn economic evaluation and reserves estimation
- **Notebook 03**: Analyze multiple wells simultaneously
- **Notebook 04**: Explore advanced ML forecasting models